In [1]:
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

import optuna
import lightgbm as lgb

import pickle

In [2]:
# !gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_02_fe_v01.parquet /home/eanegrin/datasets/

In [2]:
base_path = 'C:/Eugenio/Maestria/DMEyF/'
# base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_03_fe_v05_undersampled.parquet'

ganancia_acierto = 273000
costo_estimulo = 7000

semillas = [122219, 109279, 400391, 401537, 999961]

In [3]:
# data = pd.read_parquet('/home/eanegrin/datasets/' + dataset_file)
data = pd.read_parquet(dataset_path + dataset_file)

In [4]:
# para recordar los periodos en los que entrenamos el modelo final:
data['foto_mes'].unique()

array([201906, 201907, 201908, 201909, 201910, 201911, 201912, 202001,
       202002, 202003, 202004, 202005, 202007, 202008, 202009, 202010,
       202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106,
       202107])

In [5]:
data.shape

(206434, 1278)

In [6]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [7]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)

In [8]:
X_train = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_train_binaria = data['clase_binaria'] # Junta a los 2 baja
w_train = data['clase_peso']

In [9]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Entrenamiento

In [10]:
storage_name = "sqlite:///" + db_path + "optimization_lgbm_v05.db"
study_name = "competencia3_lgbm_v05" # UPDATE

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-12-03 09:48:04,785] Using an existing study with name 'competencia3_lgbm_v05' instead of creating a new one.


In [11]:
resultados = study.trials_dataframe()
resultados.shape

(85, 12)

In [12]:
study.best_trial.params

{'num_leaves': 243,
 'learning_rate': 0.016794504627976162,
 'min_data_in_leaf': 866,
 'feature_fraction': 0.7678004709395572,
 'bagging_fraction': 0.5901190076375339}

Reentrenamos los modelos individuales con la totalidad de los datos:

In [13]:
version = 'v005' # UPDATE

best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model {best_iter}")

counter = 0

for semilla in semillas:
    
    counter += 1
    print(f'{counter}. Train del modelo {version} con semilla {semilla}')    
    
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': study.best_trial.params['num_leaves'],
        'learning_rate': study.best_trial.params['learning_rate'],
        'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
        'feature_fraction': study.best_trial.params['feature_fraction'],
        'bagging_fraction': study.best_trial.params['bagging_fraction'],
        'seed': semilla,
        'verbose': -1
    }

    train_data = lgb.Dataset(X_train,
                            label=y_train_binaria,
                            weight=w_train)

    model = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)
    
    model.save_model(modelos_path + f'{version}/lgb_competencia3_{version}_s{semilla}_final.txt') # _final para indicar que es la version entrenada con todo el conjunto disponible.

Mejor cantidad de árboles para el mejor model 621
1. Train del modelo v005 con semilla 122219
2. Train del modelo v005 con semilla 109279
3. Train del modelo v005 con semilla 400391
4. Train del modelo v005 con semilla 401537
5. Train del modelo v005 con semilla 999961
